# Pre and Compare sampling mthod

In [67]:
import math
import pdb
import warnings
warnings.filterwarnings("ignore")
import time
from skmultilearn.model_selection import IterativeStratification
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelD
from skmultilearn.adapt import MLkNN
from skmultilearn.adapt import BRkNNbClassifier
from sklearn.neighbors import NearestNeighbors
from enum import Enum
from skmultilearn.dataset import load_dataset
from skmultilearn.dataset import load_from_arff
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB
from skmultilearn.ensemble import RakelO
from skmultilearn.adapt import MLTSVM
from skmultilearn.adapt import MLARAM
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from scipy import sparse
import random
from collections import Counter
def CardAndDens(X,y):
    cardmatrix=[]
    for i in range(X.shape[0]):
        count=0
        for j in range(y.shape[1]):
            if y[i,j]==1:
                count+=1
        cardmatrix.append(count)
    Card=sum(cardmatrix)/len(cardmatrix)
    Dens=Card/y.shape[1]
    return Card,Dens
def FeatureSelect(p):
    if p==1:
        return X.toarray(),feature_names
    else:
        featurecount=int(X.shape[1]*p)
        Selectfeatureindex=[x[0] for x in (sorted(enumerate(X.sum(axis=0).tolist()[0]),key=lambda x: x[1],reverse=True))][:featurecount]
        Allfeatureindex=[i for i in range(X.shape[1])]
        featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
        new_x=np.delete(X.toarray(),featureindex,axis=1)
        new_featurename=[feature_names[i] for i in Selectfeatureindex] 
        return new_x,new_featurename
def ImR(X,y):
    Imr=[]
    for i in range(y.shape[1]):
        count0=0
        count1=0
        for j in range(y.shape[0]):
            if y[j,i]==1:
                count1+=1
            else:
                count0+=1
        if count1<=count0:
            Imr.append(count0/count1)
        else:
            Imr.append(count1/count0)
    return Imr
def Imbalance(X,y):
    countmatrix=[]
    for i in range(y.shape[1]):
        count0=0
        count1=0
        for j in range(y.shape[0]):
            if y[j,i]==1:
                count1+=1
            else:
                count0+=1
        countmatrix.append(count1)
    maxcount=max(countmatrix)
    ImbalanceRatioMatrix=[maxcount/i for i in countmatrix]
    MaxIR=max(ImbalanceRatioMatrix)
    MeanIR=sum(ImbalanceRatioMatrix)/len(ImbalanceRatioMatrix)
    return ImbalanceRatioMatrix,MeanIR,countmatrix
def Scumble(X,y):
    ImbalanceRatioMatrix,MeanIR,_=Imbalance(X,y)
    DifferenceImbalanceRatioMatrix=[i-MeanIR for i in ImbalanceRatioMatrix]
    count=0
    for i in range(y.shape[1]):
        count+=math.pow(DifferenceImbalanceRatioMatrix[i],2)/(y.shape[1]-1)
    ImbalanceRatioSigma=math.sqrt(count)
    CVIR=ImbalanceRatioSigma/MeanIR
    SumScumble=0
    Scumble_i=[]
    for i in range(y.shape[0]):
        count=0
        prod=1
        SumIRLbl=0
        for j in range(y.shape[1]):
            IRLbl=1
            if y[i,j]==1:
                IRLbl=ImbalanceRatioMatrix[j]
                SumIRLbl+=IRLbl
                prod*=IRLbl
                count+=1
        if count==0:
            Scumble_i.append(0)
        else:
            IRLbl_i=SumIRLbl/count
            Scumble_i.append(1.0-((1.0/IRLbl_i) * math.pow(prod, 1.0/count)))
    scumble=sum(Scumble_i)/X.shape[0]
    return Scumble_i,scumble,CVIR
def Labeltype(X,y):
    ImbalanceRatioMatrix,MeanIR,_=Imbalance(X,y)
    DifferenceImbalanceRatioMatrix=[i-MeanIR for i in ImbalanceRatioMatrix]
    MinLabelIndex=[]
    MajLabelIndex=[]
    count=0
    for i in (DifferenceImbalanceRatioMatrix):
        if i>0:
            MinLabelIndex.append(count)
        else:
            MajLabelIndex.append(count)
        count+=1
    MinLabelName=[]
    MajLabelName=[]
    for i in MinLabelIndex:
        MinLabelName.append(label_names[i][0])
    for i in MajLabelIndex:
        MajLabelName.append(label_names[i][0])
    MinLabeldic=dict(zip(MinLabelIndex,MinLabelName))
    MajLabeldic=dict(zip(MajLabelIndex,MajLabelName))
    return MinLabeldic,MajLabeldic
def CalcuNN(df1,n_neighbor):
    nbs=NearestNeighbors(n_neighbors=n_neighbor,metric='euclidean',algorithm='kd_tree').fit(df1)
    euclidean,indices= nbs.kneighbors(df1)
    return euclidean,indices
def MLSMOTE(df1,df2,n_neighbor,method):
    start_time = time.time()
    ML_SMOTE_new_X=df1.copy(deep=True)
    ML_SMOTE_target=df2.copy(deep=True)
    MinLabeldic,MajLabeldic=Labeltype(np.array(df1),np.array(df2))
    ImbalanceRatioMatrix,MeanIR,countmatrix=Imbalance(np.array(df1),np.array(df2))
    MinLabelindex=list(MinLabeldic.keys())
    tem=['True']*len(MinLabelindex)
    dic=dict(zip(MinLabelindex,tem))
    minlabels=[]
    for i in range(df2.shape[1]):
        count0=0
        count1=0
        for j in range(df2.shape[0]):
            if df2.iloc[j,i]==1:
                count1+=1
            else:
                count0+=1
        if count1<=count0:
            minlabels.append(1)
        else:
            minlabels.append(0)
    np.random.seed(100)
    for tail_label in MinLabelindex:
        if(dic[tail_label]=='False'):
            continue        
        new_IRLbl=max(countmatrix)/countmatrix[tail_label]
        if(new_IRLbl<=MeanIR):
            dic[tail_label]='False'
            continue  
        sub_index=list(df2[df2[MinLabeldic[tail_label]]==minlabels[tail_label]].index)
        dfX= df1[df1.index.isin(sub_index)].reset_index(drop = True)
        dfy= df2[df2.index.isin(sub_index)].reset_index(drop = True)
        new_X = np.zeros((dfX.shape[0], dfX.shape[1]))
        target = np.zeros((dfy.shape[0], dfy.shape[1]))
        count=0
        if dfX.shape[0]==1:
            new_X[count]=np.array(dfX)
            target[count]=np.array(dfy)
            count+=1
            break
        if(dfX.shape[0]>n_neighbor):
            euclidean,indices=CalcuNN(dfX,n_neighbor+1)
        else:
            euclidean,indices=CalcuNN(dfX,dfX.shape[0])
        for i in range(dfX.shape[0]):
            seed = i
            reference = np.random.choice(indices[seed,1:])
            all_point = indices[seed,:]
            nn_df = dfy[dfy.index.isin(all_point)]
            ser = nn_df.sum(axis = 0, skipna = True)      
            for j in range(dfX.shape[1]): 
                reference = np.random.choice(indices[seed,1:])
                ratio=np.random.random()
                if feature_names[j][1]=='NUMERIC':  
                    new_X[count,j] = dfX.iloc[seed,j] + ratio * (dfX.iloc[reference,j] - dfX.iloc[seed,j])
                elif feature_names[j][1]==['YES', 'NO'] or feature_names[j][1]==['0', '1']:
                    if ratio>=0.5:
                        new_X[count,j] =dfX.iloc[seed,j]
                    else:
                        new_X[count,j] =dfX.iloc[reference,j]
                else:
                    new_X[count,j] =dfX.iloc[seed,j]
            if (method=="Ranking"):
                target[count] = np.array([1 if val>=((n_neighbor+1)/2) else 0 for val in ser])
            elif (method=="Union"):
                target[count] = np.array([1 if val>0 else 0 for val in ser])
            else:   
                target[count] = np.array([1 if val==len(all_point) else 0 for val in ser]) 
            for k in MinLabelindex:
                if target[count,k]==minlabels[k]:
                    countmatrix[k]+=1
            count+=1
        new_X = pd.DataFrame(new_X,columns=[x[0] for x in feature_names])
        target = pd.DataFrame(target,columns=[y[0] for y in label_names])
        dropindex=[]
        for i in range(target.shape[0]):
            d=np.array(df2.iloc[i:])
            if d.sum()==0:
                dropindex.append(i)
        new_X.drop(dropindex).reset_index(drop=True)
        target.drop(dropindex).reset_index(drop=True)
        ML_SMOTE_new_X = pd.concat([ML_SMOTE_new_X, new_X], axis=0).reset_index(drop=True)
        ML_SMOTE_target = pd.concat([ML_SMOTE_target, target], axis=0).reset_index(drop=True)
    end_time = time.time()
    times=end_time-start_time
    return ML_SMOTE_new_X,ML_SMOTE_target
class LocalType(Enum):
    Safe=1
    BoderLine=2
    Rare=3
    Outlier=4
    Majority=0
def MLSOL(df1,df2,p):
    start_time = time.time()
    np.random.seed(21)
    n_neighbors=5
    nbs=NearestNeighbors(n_neighbors=n_neighbors+1,metric='euclidean',algorithm='kd_tree').fit(df1)
    euclidean2,indices2= nbs.kneighbors(df1)
    minlabels=[]
    for i in range(df2.shape[1]):
        count0=0
        count1=0
        for j in range(df2.shape[0]):
            if df2.iloc[j,i]==1:
                count1+=1
            else:
                count0+=1
        if count1<=count0:
            minlabels.append(1)
        else:
            minlabels.append(0)      
    C=np.zeros((df2.shape[0],df2.shape[1]))
    for i in range(df2.shape[0]): 
        all_neighbour = indices2[i,1:]
        for j in range(df2.shape[1]):
            if df2.iloc[i,j]==minlabels[j]:
                count=0
                for k in all_neighbour.tolist():
                    if df2.iloc[i,j]!=df2.iloc[k,j]:
                        count+=1
                C[i,j]=(count)/n_neighbors
            else:
                C[i,j]=0
    W=np.zeros(df1.shape[0])
    tem=np.zeros([df2.shape[0],df2.shape[1]])
    for j in range(df2.shape[1]):    
        SumC=0.0
        c=0
        for i in range(df2.shape[0]):
            if C[i,j]<1 and C[i,j]!=0:
                SumC+=C[i,j]
                c+=1
        if SumC!=0.0 and c!=0:
            for i in range(df2.shape[0]):
                if C[i,j]<1 and C[i,j]!=0:
                    tem[i,j]=C[i,j]/SumC
        else:
            tem[i,j]=0
    SumW=0
    for i in range(df2.shape[0]):
        for j in range(df2.shape[1]):
            if tem[i,j]!=0:
                W[i]+=tem[i,j]
        SumW+=W[i]
    InstanceType=np.zeros([df2.shape[0],df2.shape[1]])
    for i in range(df2.shape[0]):
        for j in range(df2.shape[1]):
            if df2.iloc[i,j]==minlabels[j]:
                if C[i,j]<0.3:
                    InstanceType[i,j]=LocalType.Safe.value
                elif C[i,j]<0.7:
                    InstanceType[i,j]=LocalType.BoderLine.value
                elif C[i,j]<1:
                    InstanceType[i,j]=LocalType.Rare.value
                elif C[i,j]==1:
                    InstanceType[i,j]=LocalType.Outlier.value
# majority class
            else:
                InstanceType[i,j]=LocalType.Majority.value 
# reanalyse
#     for i in range(df2.shape[0]):
#         for j in range(df2.shape[1]):
#             if InstanceType[i,j]==LocalType.Rare.value:
#                 for k in indices2[i,1:]:
#                     if (InstanceType[k,j]==LocalType.Safe.value or InstanceType[k,j]==LocalType.Safe.value):
#                         InstanceType[i,j]=LocalType.BoderLine.value
#                         break                   
    n_sample=int(df1.shape[0]*p)
    new_X = np.zeros((n_sample, df1.shape[1]))
    target = np.zeros((n_sample, df2.shape[1]))
    for i in range(n_sample):
        random_count=np.random.random()*SumW
        seed=0
        s=0
        for k in range(len(W)):
            s+=W[k]
            if(random_count<=s):
                seed=k
                break            
   #     生成样本空间
        for j in range(df1.shape[1]): 
    #         随机选取一个近邻
            reference = np.random.choice(indices2[seed,1:])
            ratio=np.random.random()
            if feature_names[j][1]=='NUMERIC':  
                new_X[i,j] = df1.iloc[seed,j] + ratio*(df1.iloc[reference,j]-df1.iloc[seed,j])
            elif feature_names[j][1]==['YES', 'NO'] or feature_names[j][1]==['0', '1']:
                rmd=np.random.choice([True, False])
                if rmd==True:
                    new_X[i,j]=df1.iloc[seed,j] 
                else:
                    new_X[i,j]=df1.iloc[reference,j]
            else:
                new_X[i,j]=df1.iloc[seed,j]
    #     生成样本和种子实例的欧氏距离
        dist1 = np.linalg.norm(new_X[i]-np.array(df1.loc[seed,:]))
    #     生成样本和近邻实例的欧氏距离
        dist2 = np.linalg.norm(new_X[i]-np.array(df1.loc[reference,:]))
    #     计算cd
        cd=dist1/(dist1+dist2)
        npseed=np.array(df2.loc[seed])
        npreference=np.array(df2.loc[reference])
#     生成标签空间
        for j in range(df2.shape[1]):
            if npseed[j]==npreference[j]:
                target[i,j]=df2.iloc[seed,j]
            else:
                theta=0.5
                if InstanceType[seed,j]==LocalType.Majority.value:
                    npseed[j],npreference[j]=npreference[j],npseed[j]
                    cd=1-cd
                else:
                    if InstanceType[seed,j]==LocalType.Safe.value:
                        theta=0.5
                    elif InstanceType[seed,j]==LocalType.BoderLine.value:
                        theta=0.75
                    elif InstanceType[seed,j]==LocalType.Rare.value:
                        theta=1+(1e-5)
                    else:
                        theta=0-(1e-5)
                if cd<=theta:
                    target[i,j]=npseed[j]
                else:
                    target[i,j]=npreference[j]
    new_X = pd.DataFrame(new_X,columns=[x[0] for x in feature_names])
    target = pd.DataFrame(target,columns=[y[0] for y in label_names])
    ML_SOL_new_X = pd.concat([df1, new_X], axis=0).reset_index(drop=True)
    ML_SOL_target = pd.concat([df2, target], axis=0).reset_index(drop=True)
    end_time = time.time()
    times=end_time-start_time
    return ML_SOL_new_X,ML_SOL_target
def MLROS(df1,df2,p):
    start_time = time.time()
    np.random.seed(10)
    SamplesToClone=int(df1.shape[0]*p)
    sub_index=list()
    ImbalanceRatioMatrix,MeanIR,countmatrix=Imbalance(np.array(df1),np.array(df2))
    MinLabeldic,_=Labeltype(np.array(df1),np.array(df2))
    ML_ROS_new_X=df1.copy(deep=True)
    ML_ROS_target=df2.copy(deep=True)
    MinLabelindex=list(MinLabeldic.keys())
    tem=['True']*len(MinLabelindex)
    dic=dict(zip(MinLabelindex,tem))
    minlabels=[]
    for i in range(df2.shape[1]):
        count0=0
        count1=0
        for j in range(df2.shape[0]):
            if df2.iloc[j,i]==1:
                count1+=1
            else:
                count0+=1
        if count1<=count0:
            minlabels.append(1)
        else:
            minlabels.append(0)      
    while(SamplesToClone>0):
        for tail_label_index in MinLabelindex:
            if(dic[tail_label_index]=='False'):
                continue        
            new_IRLbl=max(countmatrix)/countmatrix[tail_label_index]
            if(new_IRLbl<=MeanIR):
                dic[tail_label_index]='False'
                continue  
            sub_index=df2[df2[MinLabeldic[tail_label_index]]==minlabels[tail_label_index]].index
            randomindex=np.random.choice(sub_index)
            for index in MinLabelindex:
                if(df2.iloc[randomindex,index]==minlabels[tail_label_index]):
                    countmatrix[index]+=1
            ML_ROS_new_X = pd.concat([ML_ROS_new_X, df1.loc[[randomindex]]], axis=0).reset_index(drop=True)
            ML_ROS_target = pd.concat([ML_ROS_target, df2.loc[[randomindex]]], axis=0).reset_index(drop=True)
            SamplesToClone=SamplesToClone-1
            if(SamplesToClone<=0):
                break  
        if(set(dic.values())==({'False'})):
            break
    end_time = time.time()
    times=end_time-start_time
    return ML_ROS_new_X,ML_ROS_target
def MLRUS(df1,df2,p):
    start_time = time.time()
    np.random.seed(10)
    SamplesToDelete=int(df1.shape[0]*p)
    sub_index=list()
    DeleteIndex=list()
    ImbalanceRatioMatrix,MeanIR,countmatrix=Imbalance(np.array(df1),np.array(df2))
    MinLabeldic,MajLabeldic=Labeltype(np.array(df1),np.array(df2))
    MinLabelindex=list(MinLabeldic.keys())
    minlabels=[]
    for i in range(df2.shape[1]):
        count0=0
        count1=0
        for j in range(df2.shape[0]):
            if df2.iloc[j,i]==1:
                count1+=1
            else:
                count0+=1
        if count1<=count0:
            minlabels.append(1)
        else:
            minlabels.append(0)     
    protectins=[]
    for min_index in MinLabelindex:
        if df2.iloc[:,min_index].sum()==1:
            for j in range(df2.shape[0]):
                if df2.iloc[j,min_index]==1:
                    protectins.append(j) 
    ML_RUS_new_X=df1.copy(deep=True)
    ML_RUS_target=df2.copy(deep=True)
    MajLabelindex=list(MajLabeldic.keys())
    tem=['True']*len(MajLabelindex)
    dic=dict(zip(MajLabelindex,tem))
    while(SamplesToDelete>0):
        for maj_index in MajLabelindex:
            if(dic[maj_index]=='False'):
                continue        
            new_IRLbl=max(countmatrix)/countmatrix[maj_index]
            if(new_IRLbl>MeanIR):
                dic[maj_index]='False'
                continue  
            sub_index=df2[df2[MajLabeldic[maj_index]]==minlabels[maj_index]].index
            randomindex=np.random.choice(sub_index)
            if(randomindex in DeleteIndex) or (randomindex in protectins):
                continue
            DeleteIndex.append(randomindex)
            for index in MajLabelindex:
                if(df2.iloc[randomindex,index]==minlabels[maj_index]):
                    countmatrix[index]-=1
            SamplesToDelete=SamplesToDelete-1
            if(SamplesToDelete<=0):
                break  
        if(set(dic.values())==({'False'})):
            break
    ML_RUS_new_X = ML_RUS_new_X.drop(index=DeleteIndex).reset_index(drop=True)
    ML_RUS_target = ML_RUS_target.drop(index=DeleteIndex).reset_index(drop=True)
    end_time = time.time()
    times=end_time-start_time
    return ML_RUS_new_X,ML_RUS_target
def REMEDIAL(df1,df2):
    Scumble_i,scumble,_=Scumble(np.array(df1),np.array(df2))
    MinLabeldic,MajLabeldic=Labeltype(np.array(df1),np.array(df2))
    allindex=[i for i,scumble_i in enumerate(Scumble_i) if scumble_i>scumble]
    REMEDIAL_X=df1.copy(deep=True)
    REMEDIAL_target=df2.copy(deep=True)
    change_x=df1.iloc[allindex]
    change_y=df2.iloc[allindex]
    for i in allindex:
        for tail_label in MinLabeldic.values():
            REMEDIAL_target.loc[i,tail_label]=0        
        for maj_label in MajLabeldic.values():
            change_y.loc[i,maj_label]=0
    new_X=pd.concat([REMEDIAL_X,change_x],axis=0).reset_index(drop=True)
    new_target=pd.concat([REMEDIAL_target,change_y],axis=0).reset_index(drop=True)
    return new_X,new_target
def diflabel(df1,df2):
    nbs=NearestNeighbors(n_neighbors=df1.shape[0],metric='euclidean',algorithm='kd_tree').fit(df1)
    euclidean,indices= nbs.kneighbors(df1)
    cnt=[]
    for i in range(df1.shape[0]):
        count=0
        for j in indices[i,1:6]:
            if np.array_equal(df2.iloc[i], df2.iloc[j]):
                count+=1
        cnt.append(count/5)
    print(sum(cnt)/len(cnt))
    print(sum((x -(sum(cnt)/len(cnt))) ** 2 for x in cnt) / len(cnt))

# Compare base classifier

In [68]:
import torch
import torch.nn.functional as F
from skmultilearn.dataset import load_dataset
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import hamming_loss, accuracy_score, f1_score, precision_score, recall_score
import pdb
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
class Fd(torch.nn.Module):
    """
    Simple fully connected network.
    """

    def __init__(self, in_dim, H, out_dim, fin_act=None):
        super(Fd, self).__init__()
        self.fc1 = torch.nn.Linear(in_dim, H)
        self.fc2 = torch.nn.Linear(H, out_dim)
        self.fin_act = fin_act

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = self.fc2(x)
        return self.fin_act(x) if self.fin_act else x
class Fx(torch.nn.Module):
    def __init__(self, in_dim, H1, H2, out_dim):
        super(Fx, self).__init__()
        self.fc1 = torch.nn.Linear(in_dim, H1)
        self.fc2 = torch.nn.Linear(H1, H2)
        self.fc3 = torch.nn.Linear(H2, out_dim)
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.leaky_relu(self.fc3(x))
        return x
class Fe(torch.nn.Module):

    def __init__(self, in_dim, H, out_dim):
        super(Fe, self).__init__()
        self.fc1 = torch.nn.Linear(in_dim, H)
        self.fc2 = torch.nn.Linear(H, out_dim)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        return x
# C2AE
class C2AE(torch.nn.Module):

    def __init__(self, Fx, Fe, Fd, beta=1, alpha=.5, emb_lambda=.5, latent_dim=6,
                 device=None):
        super(C2AE, self).__init__()
        # Define main network components.
        # Encodes x into latent space. X ~ z
        self.Fx = Fx
        # Encodes y into latent space. Y ~ z
        self.Fe = Fe
        # Decodes latent space into Y. z ~ Y
        self.Fd = Fd

        # Hyperparam used to set tradeoff between latent loss, and corr loss.
        self.alpha = alpha
        self.beta = beta
        # Lagrange to use in embedding loss.
        self.emb_lambda = emb_lambda
        self.latent_I = torch.eye(latent_dim).to(device)

    def forward(self, x, y=None):
        if self.training:
            # Calculate feature, and label latent representations.
            fx_x = self.Fx(x)
            fe_y = self.Fe(y)
            # Calculate decoded latent representation.
            fd_z = self.Fd(fe_y)
            return fx_x, fe_y, fd_z
        else:
            # If evaluating just send through encoder and decoder.
            return self.predict(x)

    def _predict(self, y):
        return self.Fd(self.Fe(y))

    def predict(self, x):
        return self.Fd(self.Fx(x))

    def corr_loss(self, preds, y):
        # Generate masks for [0,1] elements.
        ones = (y == 1)
        zeros = (y == 0)
        # Use broadcasting to apply logical and between mask arrays.
        # This will only indicate locations where both masks are 1.
        # THis corresponds to set we are enumerating in eq (3) in Yah et al.
        ix_matrix = ones[:, :, None] & zeros[:, None, :]
        # Use same broadcasting logic to generate exponetial differences.
        # This like the above broadcast will do so between all pairs of points
        # for every datapoint.
        diff_matrix = torch.exp(-(preds[:, :, None] - preds[:, None, :]))
        # This will sum all contributes to loss for each datapoint.
        losses = torch.flatten(diff_matrix*ix_matrix, start_dim=1).sum(dim=1)
        # Normalize each loss add small epsilon incase 0.
        losses /= (ones.sum(dim=1)*zeros.sum(dim=1) + 1e-4)
        # Replace inf, and nans with 0.
        losses[losses == float('Inf')] = 0
        losses[torch.isnan(losses)] = 0
        # Combine all losses to retrieve final loss.
        return losses.sum()

    def latent_loss(self, fx_x, fe_y):
        c1 = fx_x - fe_y
        # Here to help hold constraint of FxFx^2 = FyFy^2 = I.
        c2 = fx_x.T@fx_x - self.latent_I
        c3 = fe_y.T@fe_y - self.latent_I
        # Combine loss components as specified in Yah et al.
        latent_loss = torch.trace(
            c1@c1.T) + self.emb_lambda*torch.trace(c2@c2.T + c3@c3.T)
        # ********** Version 2: Ignore constraint **********
        #latent_loss = torch.mean((fx_x - fe_y)**2)
        return latent_loss
    def losses(self, fx_x, fe_y, fd_z, y):
        l_loss = self.latent_loss(fx_x, fe_y)
        c_loss = self.corr_loss(fd_z, y)
        return l_loss, c_loss   
def save_model(model, path):
    torch.save(model.state_dict(), path)
def load_model(model_cls, path, *args, **kwargs):
    model = model_cls(*args, **kwargs)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model
def eval_metrics(mod, metrics, datasets, device, apply_sig=False):
    res_dict = {}
    for ix, dataset in enumerate(datasets):
        mod.eval()
        x = dataset.tensors[0].to(device).float()
        # Make predictions.
        preds = mod(x)
        y_prob = preds.cpu().detach().numpy()
        # Convert them to binary multilabels.
        if apply_sig:
            y_pred = torch.round(torch.sigmoid(preds)).cpu().detach().numpy()
        else:
            y_pred = torch.round(preds).cpu().detach().numpy()
        y_true = dataset.tensors[1].cpu().detach().numpy()
        # Calculate metric.
        res_dict[f'dataset_{ix}'] = {metric.__name__: metric(y_true, y_pred) for metric in metrics}
    return res_dict
# COCOA
class COCOA:
    np.random.seed(10)
    def __init__(self):
        self.classifiers = []
        self.thresholds = []
        self.mclassifiers=[]
        self.mthresholds=[]
        self.couples=3
        self.classname=np.array([0,1,2])      
    def find_optimal_threshold(self, X, Y):
        self.classifiers = []
        self.thresholds = []
        num_labels = Y.shape[1]
        for i in range(num_labels):
            y = Y[:, i]
            clf = LogisticRegression()
            clf.fit(X, y)
            self.classifiers.append(clf)       
            y_pred = clf.predict_proba(X)[:, 1]
            f1_scores = []
            thresholds = np.arange(0, 1.01, 0.01)
            for threshold in thresholds:
                y_pred_binary = (y_pred >= threshold).astype(int)
                f1 = f1_score(y, y_pred_binary, average='binary')
                f1_scores.append(f1)
            best_threshold_idx = np.argmax(f1_scores)
            self.thresholds.append(thresholds[best_threshold_idx])
    def fit(self, X, Y):
        self.find_optimal_threshold(X, Y)
        self.mclassifiers = []
        couples=self.couples
        num_labels = Y.shape[1]
        for i in range(num_labels):
            new_y=self.multilabel_to_multiclass(Y,i)
            clf = LogisticRegression()
            clf.fit(X, new_y)
            self.mclassifiers.append(clf)
    def multilabel_to_multiclass(self, y, index):
        np.random.seed(10)
        couples=self.couples
        classname=self.classname
        new_y = np.zeros((y.shape[0],))
        other_labels = np.delete(np.arange(y.shape[1]), index)
        couple_labels = np.random.choice(other_labels, couples, replace=False)
        index_label = y[:, index]     
        new_y[index_label == 1] = classname[2]
        couple_labels_present = (y[:, couple_labels] == 1).any(axis=1)
        new_y[(index_label == 0) & couple_labels_present] = classname[1]
        new_y[(index_label == 0) & (~couple_labels_present)] = classname[0]
        return new_y.astype(int)
    def predict(self, X):
        num_samples = X.shape[0]
        classname=self.classname
        Y_pred = np.zeros((num_samples, len(self.mclassifiers)))        
        for i, clf in enumerate(self.mclassifiers):
            y_pred = clf.predict_proba(X)[:, -1]
            Y_pred[:, i] = (y_pred >= self.thresholds[i]).astype(int)
        return Y_pred 
    def predict_proba(self, X):
        num_samples = X.shape[0]
        classname=self.classname
        Y_pred = np.zeros((num_samples, len(self.mclassifiers)))
        for i, clf in enumerate(self.mclassifiers):
            y_pred = clf.predict_proba(X)[:, -1]
            Y_pred[:, i] = y_pred
        return Y_pred
class ECC:
    def __init__(self):
        self.num_classifiers = 10
        self.labels = y.shape[1]
        self.k =y.shape[1]
        self.classifiers = []
        self.classifiers_label_index = []
        for _ in range(self.num_classifiers):
            classifier = ClassifierChain(
                classifier=XGBClassifier(random_state=42, n_jobs=-1, eta=0.05, max_depth=7, colsample_bytree=1,
                                         n_estimators=377,
                                         tree_method='gpu_hist'),
                require_dense=[False, True]
            )
            self.classifiers.append(classifier)

    def fit(self, X, Y):
        num_instances = X.shape[0]
        num_labels = Y.shape[1]
        original_list =list(range(self.labels))
        for classifier in self.classifiers:
            random_sorted_list = np.random.permutation(original_list)
            self.classifiers_label_index.append(random_sorted_list)
            selected_instances = np.random.choice(num_instances, size=int(1 * num_instances), replace=False)
            classifier.fit(X[selected_instances], Y[selected_instances][:, random_sorted_list])

    def predict(self, X, threshold=0.5):
        probabilities = self.predict_proba(X)
        Y_pred = (probabilities >= threshold).astype(int)
        return Y_pred
    def predict_proba(self, X):
        Y_pred = np.zeros((X.shape[0], self.labels))
        i = 0
        for classifier in self.classifiers:
            proba_matrix = classifier.predict_proba(X)
            for j, label in enumerate(self.classifiers_label_index[i]):
                Y_pred[:, label] += proba_matrix[:, j].toarray().squeeze()
            i = i + 1
        Y_pred /= self.num_classifiers
        return Y_pred

# GET W

In [69]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
# Caculate W
def CLML(X, Y, optmParameter):
    # optimization parameters
    alpha = optmParameter['alpha']
    beta = optmParameter['beta']
    gamma = optmParameter['gamma']
    lamda = optmParameter['lamda']
    lamda2 = optmParameter['lamda2']
    maxIter = optmParameter['maxIter']
    miniLossMargin = optmParameter['minimumLossMargin']
    
    # initialization
    num_dim = X.shape[1]
    XT = X.T
    XTX = np.dot(XT, X)
    XTY = np.dot(XT, Y)
    W_s = np.linalg.solve(XTX + gamma*np.eye(num_dim), XTY)
    W_s_1 = W_s
    
    # label correlation
    R = pdist(Y.T+np.finfo(float).eps, metric='cosine')
    R = 1 - squareform(R)
    C = R.reshape(Y.shape[1], Y.shape[1])
    L1 = np.diag(np.sum(C, axis=1)) - C
    
    S = ins_similarity(X, 5)
    L2 = np.diag(np.sum(S, axis=1)) - S
    iter = 1
    oldloss = 0
    bk = 1
    bk_1 = 1
    
    # compute LIP
    A = gradL21(W_s)
#     Lip = np.sqrt(4*(np.linalg.norm(XTX)**2 + np.linalg.norm(alpha*XTX)**2 * np.linalg.norm(L1)**2 
#                   + np.linalg.norm(lamda*XT.dot(L2).dot(X))**2) + np.linalg.norm(lamda2*A)**2)
    Lip = np.sqrt(4*(np.linalg.norm(XTX)**2 + np.linalg.norm(alpha*XTX)**2 * np.linalg.norm(L1)**2))   
    # proximal gradient
    while iter <= maxIter:
        A = gradL21(W_s)
        W_s_k = W_s + (bk_1 - 1)/bk * (W_s - W_s_1)
#         gradF = np.dot(XTX, W_s_k) - XTY + alpha * np.dot(XTX, W_s_k).dot(L1) + lamda * XT.dot(L2).dot(X).dot(W_s_k) + lamda2*A.dot(W_s_k)
        gradF = np.dot(XTX, W_s_k) - XTY + alpha * np.dot(XTX, W_s_k).dot(L1) 
        Gw_s_k = W_s_k - 1/Lip *(gradF)
        # update b, W
        bk_1 = bk
        bk = (1 + np.sqrt(4*bk**2 + 1))/2
        W_s_1 = W_s
        W_s = softthres(Gw_s_k, beta/Lip)
        
        # compute loss function
        predictionLoss = np.trace(np.dot(X.dot(W_s) - Y, (X.dot(W_s) - Y).T))
        F = X.dot(W_s)
        correlation = np.trace(F.dot(L1).dot(F.T))
        In_correlation = np.trace(F.T.dot(L2).dot(F))
        sparsity = np.sum(W_s != 0)
        sparsity2 = np.trace(np.dot(W_s.T, A).dot(W_s))
        totalloss = predictionLoss + alpha*correlation + beta*sparsity 
#         totalloss = predictionLoss + alpha*correlation + beta*sparsity + lamda*In_correlation + lamda2*sparsity2
        if abs(oldloss - totalloss) <= miniLossMargin or totalloss <= 0:
            break
        else:
            oldloss = totalloss
        iter += 1
    
    model_NewLLSF = W_s
    return model_NewLLSF
def softthres(W_t, lambda_):
    return np.maximum(W_t-lambda_, 0) - np.maximum(-W_t-lambda_, 0)
def ins_similarity(X, K):
    A = squareform(pdist(X))
    num_dim = A.shape[0]
    for i in range(num_dim):
        temp = A[i,:]
        As = np.sort(temp)
        temp = (temp <= As[K])
        A[i,:] = temp
    return A
def label_similarity(Y,k):
    m = Y.shape[1]
    cos_sim = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            if i == j:
                cos_sim[i, j] = 1.0
            else:
                cos_sim[i, j] = np.dot(Y[:, i], Y[:, j]) / (np.linalg.norm(Y[:, i]) * np.linalg.norm(Y[:, j]))
    upper_triangle_indices = np.triu_indices(cos_sim.shape[1], k=1)
    upper_triangle_values = cos_sim [upper_triangle_indices]
    top_k_indices = np.argpartition(upper_triangle_values, -k)[-k:]
    result_matrix = np.zeros_like(cos_sim)
    result_matrix[upper_triangle_indices[0][top_k_indices], upper_triangle_indices[1][top_k_indices]] = upper_triangle_values[top_k_indices]
    return result_matrix
def gradL21(W):
    num = W.shape[0]
    D = np.zeros((num, num))
    for i in range(num):
        temp = np.linalg.norm(W[i,:], 2)
        if temp != 0:
            D[i,i] = 1/temp
        else:
            D[i,i] = 0
    return D
def TuneThreshold(output, target, bAllOne, metricIndex):
    if bAllOne is None:
        bAllOne = 1
    elif metricIndex is None:
        metricIndex = 3  # use MacroF
    
    num_train,num_class= target.shape
    TotalNums = 50
    min_score = 0
    max_score = np.max(output)
    step = (max_score - min_score) / TotalNums
    tau_range = np.arange(min_score, max_score + step, step)  
    
    tau = np.zeros(num_class)  
    currentResult = tau.copy()
    for t in range(len(tau_range)):
        threshold = tau_range[t]
        if bAllOne == 1:  
# set to only one threshold for all the class labels
            thresholds = np.ones_like(output) * threshold
            predict_target = np.where(output - thresholds >= 0, 1, 0)  
            tempResult = evaluateOneMetric(target, predict_target, metricIndex)  
            if tempResult > currentResult[0]:
                currentResult[0] = tempResult
                tau[0] = threshold
        else:  
            for c in range(num_class):
                thresholds = np.ones(num_train) * threshold
#                 pdb.set_trace()
                predict_target_l = np.where(output[:,c] - thresholds >= 0, 1, 0)                
                tempResult = evaluateF1(target[:,c], predict_target_l)
                if tempResult > currentResult[c]:
                    currentResult[c] = tempResult
                    tau[c] = threshold        
    if bAllOne == 1:
        tau = np.ones(num_class) * tau[0]   
    return tau, currentResult
def evaluateF1(target, predict):
    TP = target.dot(predict.T)
    precision = TP / np.sum(predict != 0)
    recall = TP / np.sum(target != 0)
    f1 = 2 * precision * recall / (precision + recall)
    return f1
def evaluateOneMetric(target, predict_target, metric):
    Result = 0
    if metric == 1:
        HammingScore = 1 - hamming_loss(predict_target, target)
        Result = HammingScore
    elif metric == 2 or metric == 3:
        ExampleBasedAccuracy, _, _, ExampleBasedFmeasure = example_based_measure(target, predict_target)
        if metric == 2:
            Result =ExampleBasedAccuracy
        else:
            Result = ExampleBasedFmeasure
    elif metric == 4 or metric == 5:
        LabelBasedAccuracy, _, _, LabelBasedFmeasure = label_based_measure(target, predict_target)
        if metric == 4:
            Result = LabelBasedAccuracy
        else:
            Result = LabelBasedFmeasure
    elif metric == 6:
        SubsetAccuracy = subset_accuracy_evaluation(target, predict_target)
        Result = SubsetAccuracy
    return Result
def label_based_measure(test_targets, predict_targets):
    L, num_test = test_targets.shape
    test_targets = test_targets.astype(np.float64)
    predict_targets = predict_targets.astype(np.float64)
    LabelBasedAccuracy = 0
    LabelBasedPrecision = 0
    LabelBasedRecall = 0
    LabelBasedFmeasure = 0
    for i in range(L):
        intersection = np.dot(test_targets[i,:], predict_targets[i,:])
        union = np.sum(np.logical_or(test_targets[i,:], predict_targets[i,:]))

        if union != 0:
            LabelBasedAccuracy += intersection / union

        if np.sum(predict_targets[i,:]) != 0:
            precision_i = intersection / np.sum(predict_targets[i,:])
        else:
            precision_i = 0

        if np.sum(test_targets[i,:]) != 0:
            recall_i = intersection / np.sum(test_targets[i,:])
        else:
            recall_i = 0

        LabelBasedPrecision += precision_i
        LabelBasedRecall += recall_i

        if recall_i != 0 or precision_i != 0:
            LabelBasedFmeasure += 2 * recall_i * precision_i / (recall_i + precision_i)

    LabelBasedAccuracy /= L
    LabelBasedPrecision /= L
    LabelBasedRecall /= L
    LabelBasedFmeasure /= L
    return LabelBasedAccuracy, LabelBasedPrecision, LabelBasedRecall, LabelBasedFmeasure
def example_based_measure(test_targets, predict_targets):

    L, num_test = test_targets.shape
    test_targets = np.double(test_targets == 1)
    predict_targets = np.double(predict_targets == 1)

    ExampleBasedAccuracy = 0
    ExampleBasedPrecision = 0
    ExampleBasedRecall = 0
    ExampleBasedFmeasure = 0

    for i in range(num_test):
        intersection = np.dot(test_targets[:, i], predict_targets[:, i])
        union = np.sum(np.logical_or(test_targets[:, i], predict_targets[:, i]))

        if union != 0:
            ExampleBasedAccuracy += intersection / union

        if np.sum(predict_targets[:, i]) != 0:
            precision_i = intersection / np.sum(predict_targets[:, i])
        else:
            precision_i = 0

        if np.sum(test_targets[:, i]) != 0:
            recall_i = intersection / np.sum(test_targets[:, i])
        else:
            recall_i = 0

        ExampleBasedPrecision += precision_i
        ExampleBasedRecall += recall_i

        if recall_i != 0 or precision_i != 0:
            ExampleBasedFmeasure += 2 * recall_i * precision_i / (recall_i + precision_i)
    ExampleBasedAccuracy /= num_test
    ExampleBasedPrecision /= num_test
    ExampleBasedRecall /= num_test
    ExampleBasedFmeasure /= num_test
    return ExampleBasedAccuracy, ExampleBasedPrecision, ExampleBasedRecall, ExampleBasedFmeasure
def subset_accuracy_evaluation(test_target, predict_target):
    _, num_test = test_target.shape
    count = 0
    for i in range(num_test):
        if np.array_equal(test_target[:,i], predict_target[:,i]):
            count += 1
    SubsetAccuracy = count / num_test
    return SubsetAccuracy
def Predict(Outputs, tau):
    predict_target = np.zeros_like(Outputs)  # Create a zero matrix with the same shape as Outputs
    num_class = Outputs.shape[1]  # Get the number of class labels
    for c in range(num_class):
        predict_target[:, c] = Outputs[:, c] >= tau[c]  # Copy values satisfying the tau condition to the final prediction matrix
    return predict_target

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score


# Sampling of LSDMLO

In [71]:
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances
import random
import pdb
import pickle
def CalMinkowski(df,weights,k,p1):
    n = df.shape[0]  
    distances = np.zeros((n, n))  
    data=np.array(df)
    if p1==np.inf:
        for i in range(n):
            for j in range(n):
                distances[i, j]=np.max(weights* np.abs(data[i] - data[j]))               
    else:
        for i in range(n):
            for j in range(i, n): 
                distances[i, j] = np.power(np.sum(weights * np.abs(data[i] - data[j])**p1), 1/p1)
        distances = distances + distances.T - np.diag(distances.diagonal())
    sorted_indices = np.argsort(distances, axis=1)
    sorted_indices = sorted_indices[:, ::1]
    topk_indices= sorted_indices[:, :k]
    return topk_indices
def assign_weights(arr):
    arr =np.abs(arr)
    weights = np.exp(arr - np.max(arr)) / np.sum(np.exp(arr - np.max(arr)))
#     weights =np.abs(arr)
    return weights
def weighted_minkowski_distance(v1, v2, w):
    distance = np.power(np.sum(w * np.abs(v1 - v2)**2), 1/2)
    return distance
def Imbalance(X,y):
    countmatrix=[]
    for i in range(y.shape[1]):
        count0=0
        count1=0
        for j in range(y.shape[0]):
            if y[j,i]==1:
                count1+=1
            else:
                count0+=1
        countmatrix.append(count1)
    maxcount=max(countmatrix)
    ImbalanceRatioMatrix=[maxcount/i for i in countmatrix]
    MaxIR=max(ImbalanceRatioMatrix)
    MeanIR=sum(ImbalanceRatioMatrix)/len(ImbalanceRatioMatrix)
    return ImbalanceRatioMatrix,MeanIR,countmatrix
def CalcuNN(df1,n_neighbor):
    nbs=NearestNeighbors(n_neighbors=n_neighbor,metric='euclidean',algorithm='kd_tree').fit(df1)
    euclidean,indices= nbs.kneighbors(df1)
    return euclidean,indices
def Labeltype(X,y):   
    ImbalanceRatioMatrix,MeanIR,_=Imbalance(X,y)
    DifferenceImbalanceRatioMatrix=[i-MeanIR for i in ImbalanceRatioMatrix]
    MinLabelIndex=[]
    MajLabelIndex=[]
    count=0
    for i in (DifferenceImbalanceRatioMatrix):
        if i>0:
            MinLabelIndex.append(count)
        else:
            MajLabelIndex.append(count)
        count+=1
    MinLabelName=[]
    MajLabelName=[]
    for i in MinLabelIndex:
        MinLabelName.append(label_names[i][0])
    for i in MajLabelIndex:
        MajLabelName.append(label_names[i][0])
    MinLabeldic=dict(zip(MinLabelIndex,MinLabelName))
    MajLabeldic=dict(zip(MajLabelIndex,MajLabelName))
    return MinLabeldic,MajLabeldic
def SPECIAL(df1, df2, W, sp,dataset_name):
    ImrMatrix=ImR(X,y)
    n_neighbors = 5
    p=2
    cos_sim = label_similarity(np.array(df2),10)
    non_zero_indices = [np.where(row != 0)[0].tolist() for row in cos_sim]
    row_sums = cos_sim.sum(axis=1)
    normalized_label_weight = cos_sim / row_sums[:, np.newaxis]
    MinLabeldic, MajLabeldic = Labeltype(np.array(df1), np.array(df2))
    ImbalanceRatioMatrix, MeanIR, countmatrix = Imbalance(np.array(df1), np.array(df2))
    MinLabelindex = list(MinLabeldic.keys())
    C = np.zeros((df1.shape[0], len(MinLabelindex)))
    C_hat=np.zeros((df1.shape[0], len(MinLabelindex))) 
    dataset_name = os.path.join(os.getcwd(), "rcvpara", dataset_name)
#     pdb.set_trace()
    if os.path.exists(dataset_name):
        with open(dataset_name, 'rb') as file:
            indices_dict = pickle.load(file)
        ifcalnn=False
    else:
        indices_dict = {}
        ifcalnn=True
    for tail_label in MinLabelindex:
        if ImrMatrix[tail_label]>10:
            continue
        all_relevant=non_zero_indices[tail_label]
        sub_index = np.where(df2[MinLabeldic[tail_label]] == 1)[0]
        idx = MinLabelindex.index(tail_label)
        
        W_tail_label = W[:, tail_label]
        sorted_indices = np.argsort(W_tail_label)
        sorted_column = W_tail_label[sorted_indices]
        featureWeight = assign_weights(W_tail_label)
        if ifcalnn==True:
            indices = CalMinkowski(df1, featureWeight, n_neighbors + 1, p)
#             euclidean,indices2=CalcuNN(df1,n_neighbors + 1)
            indices_dict[tail_label] = indices
        elif ifcalnn==False:
            indices=indices_dict[tail_label]
        for i in range(df1.shape[0]):
            if df2.iloc[i,tail_label]==0:
                continue
            count=0
            for j in indices[i, 1:]:
                if df2.iloc[i,tail_label]==df2.iloc[j,tail_label]:
                    count +=1
            C[i, MinLabelindex.index(tail_label)] = count / n_neighbors
            count1list=[]
            if all_relevant:   
                for k in all_relevant:
                    count1=0
                    for j in indices[i, :]:
                        if df2.iloc[j, k] == 1:
                            count1+= 1
                    count1list.append(count1)
                C_hat[i, MinLabelindex.index(tail_label)]=max(count1list) / n_neighbors      
    Ins_Weight=np.zeros(df1.shape[0])  
    tem = np.zeros([df1.shape[0], len(MinLabelindex)])
    tem_hat = np.zeros([df1.shape[0], len(MinLabelindex)])


    for j in range(len(MinLabelindex)):
        SumC = 0.0
        sum_C_1 = 0.0
        c = 0
        c_1 = 0

        for i in range(df1.shape[0]):
            if C[i, j] < 1 and C[i, j] != 0:
                SumC += C[i, j]
                c += 1
            if C_hat[i, j] != 0:
                sum_C_1 += C_hat[i, j]
                c_1 += 1

            if SumC != 0.0 and c != 0:
                if C[i, j] < 1 and C[i, j] != 0:
                    tem[i, j] = C[i, j] / SumC
            else:
                tem[i, j] = 0

            if sum_C_1 != 0.0 and c_1 != 0:
                if C_hat[i, j] != 0:
                    tem_hat[i, j] = C_hat[i, j] / sum_C_1
            else:
                tem_hat[i, j] = 0

    # check完毕，C被tem取代
    SumW = 0
    for i in range(df1.shape[0]):
        for j in range(len(MinLabelindex)):
            if tem[i, j] != 0:
                Ins_Weight[i] += tem[i, j] + tem_hat[i, j]
        SumW += Ins_Weight[i]
    non_zero_elements = []

    for row in tem:
        for element in row:
            if element != 0:
                non_zero_elements.append(element)
                
           
    n_sample = int(df1.shape[0] * sp)
    new_X = np.zeros((n_sample, df1.shape[1]))
    target = np.zeros((n_sample, df2.shape[1]))
    count = 0 
    
    while count < n_sample:  # 修正此处循环变量为count
        random_count=np.random.random()*SumW
        seed=0
        s=0
        for k in range(len(Ins_Weight)):
            s+=Ins_Weight[k]
            if(random_count<=s):
                seed=k
                break    
        seedtype = np.where(np.array(df2.iloc[seed]) == 1)[0]
        set1 = set(seedtype)
        set2 = set(MinLabelindex)
        intersection = set1.intersection(set2)
        intersection_list = list(intersection)
        if not intersection_list:
            continue  # 当 intersection_list 为空时，跳过当前循环
        select_index=np.random.choice(intersection_list)
        reference = np.random.choice(indices_dict[select_index][seed, 1:])
        all_point = indices_dict[select_index][seed, :]
        nn_df = df2[df2.index.isin(all_point)]
        ser = nn_df.sum(axis = 0, skipna = True)   
        for j in range(df1.shape[1]):
            ratio = np.random.random()
#             pdb.set_trace()
            if feature_names[j][1] == 'NUMERIC':
                new_X[count, j] = df1.iloc[seed, j] + ratio * (df1.iloc[reference, j] - df1.iloc[seed, j])
            elif feature_names[j][1] == ['YES', 'NO'] or feature_names[j][1] == ['0', '1']:
                rmd = np.random.choice([True, False])
                if rmd:
                    new_X[count, j] = df1.iloc[seed, j]
                else:
                    new_X[count, j] = df1.iloc[reference, j]
            else:
                new_X[count, j] = df1.iloc[seed, j]    
        target[count] = np.array([1 if val>=((len(all_point))/2) else 0 for val in ser])
#         for j in range(df2.shape[1]):
#             if df2.iloc[seed, j] == df2.iloc[reference, j]:
#                 target[count, j]=df2.iloc[seed, j]
#             else:
#                 featureWeight = assign_weights(W[:, j])
#                 distance1 = weighted_minkowski_distance(np.array(df1.iloc[seed, :]), new_X[count, :], featureWeight)
#                 distance2 = weighted_minkowski_distance(np.array(df1.iloc[reference, :]), new_X[count, :], featureWeight)
#                 if distance1 <= distance2:
#                     target[count, j] = df2.iloc[seed, j]
#                 else:
#                     target[count, j] = df2.iloc[reference, j]

        count += 1

    # 创建新的数据框并返回结果
#     new_X = new_X[~np.all(new_X == 0, axis=1)]
#     target= target[~np.all(target == 0, axis=1)]
    new_X = pd.DataFrame(new_X, columns=[x[0] for x in feature_names])
    target = pd.DataFrame(target, columns=[y[0] for y in label_names])
    ML_SOL_new_X = pd.concat([df1, new_X], axis=0).reset_index(drop=True)
    ML_SOL_target = pd.concat([df2, target], axis=0).reset_index(drop=True)
    if ifcalnn==True:
        with open(dataset_name, 'wb') as file:
            pickle.dump(indices_dict, file)
    return ML_SOL_new_X, ML_SOL_target

In [73]:
from skmultilearn.model_selection import IterativeStratification
from skmultilearn.adapt import MLkNN
from skmultilearn.dataset import load_dataset
from sklearn.neighbors import NearestNeighbors
import sklearn.metrics as metrics
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from sklearn.tree import DecisionTreeClassifier
from skmultilearn.dataset import load_from_arff
from skmultilearn.ensemble import RakelD
import warnings
import pandas as pd
import pdb
import numpy as np
import itertools
import os
import pickle
from xgboost import XGBClassifier
warnings.filterwarnings("ignore")
device = torch.device('cuda')
def FeatureSelect(p):
    if p==1:
        return X.toarray(),feature_names
    else:
        if feature_names[1][1]=='NUMERIC':
            featurecount=int(X.shape[1]*p)
            column_variances = np.var(X.toarray(), axis=0)
            sorted_indices = column_variances.argsort()[::-1]
            Selectfeatureindex = sorted_indices[:featurecount]
            Allfeatureindex=[i for i in range(X.shape[1])]
            featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
            new_x=np.delete(X.toarray(),featureindex,axis=1)
            new_featurename=[feature_names[i] for i in Selectfeatureindex]          
        else:
            featurecount=int(X.shape[1]*p)
            Selectfeatureindex=[x[0] for x in (sorted(enumerate(X.sum(axis=0).tolist()[0]),key=lambda x: x[1],reverse=True))][:featurecount]
            Allfeatureindex=[i for i in range(X.shape[1])]
            featureindex=[i for i in Allfeatureindex if i not in Selectfeatureindex]
            new_x=np.delete(X.toarray(),featureindex,axis=1)
            new_featurename=[feature_names[i] for i in Selectfeatureindex] 
        return new_x,new_featurename
def LabelSelect():
    b=[]
    new_labelname=[i for i in label_names]
    for i in range(y.shape[1]):
        if y[:,i].sum()<=10:
            b.append(i)
            new_labelname.remove(label_names[i])
    new_y=np.delete(y.toarray(),b,axis=1)
    return new_y,new_labelname 
# traning
def training(classiferindex,index,samplingcount,dataset_name,*args):
    Randomlist=[7,10,19,30,23]
    Macro=[]
    Micro=[]
    Hamming_loss=[]
    rankingloss=[]
    macropr=[]
    macroaucroc=[]
    Avgpr=[]
    for i in Randomlist:  
        k_fold = IterativeStratification(n_splits=2,order=1,random_state=i)
        j=0
        for train,test in k_fold.split(X,y):
            j+=1
            if classiferindex==1:
                classifier =BinaryRelevance(
                    classifier = DecisionTreeClassifier(random_state=41),
                    require_dense = [False, True]
                )
            elif classiferindex==2:
                classifier =MLkNN(k=10)
            elif classiferindex==3:
                classifier =ClassifierChain(
                    classifier = DecisionTreeClassifier(random_state=20),
                    require_dense = [False, True]
                )
            elif classiferindex==4:
                classifier = RakelD(
                    base_classifier=DecisionTreeClassifier(random_state=20),
                    base_classifier_require_dense=[False, True],
                    labelset_size=3
                )
            elif classiferindex==5:
                classifier=COCOA()
#                 classifier=ECC()
            
            if(index==1):
                X1,y1=X[train],y[train]
            else: 
                dfx=pd.DataFrame(X[train],columns=[x[0] for x in feature_names])
                dfy=pd.DataFrame(y[train],columns=[x[0] for x in label_names])
                if (index==2):
                    new_X,new_y=MLSMOTE(dfx,dfy,5,"Union")
                elif (index==3):
                    new_X,new_y=MLSMOTE(dfx,dfy,5,"Ranking")
                elif (index==4):
                    new_X,new_y=MLSOL(dfx,dfy,samplingcount)
                elif (index==5):
                    new_X,new_y=MLROS(dfx,dfy,samplingcount)
                elif (index==6):
                    new_X,new_y=MLRUS(dfx,dfy,samplingcount)
                elif(index==7):
                    new_X1,new_y1=REMEDIAL(dfx,dfy)
                    new_X,new_y=MLSMOTE(new_X1,new_y1,5,"R")
                elif(index==8):
                    new_X1,new_y1=REMEDIAL(dfx,dfy)
                    new_X,new_y=MLROS(dfx,dfy,0.1)
                elif(index==9):
                    new_X,new_y=MLBMOTE(dfx,dfy, 0.05, 0.05, 2, feature_names)
                    print(new_X.shape[0]-dfx.shape[0])                 
                else:
                    X1, y1 = X[train], y[train]
                    if j==1:
                        W = CLML(X1, y1, optmParameter)
                        W1=W
                        non_zero_counts = np.count_nonzero(W, axis=0)
                        dataset_name=dataset_name+str(j)
                        new_X, new_y = SPECIAL1(dfx, dfy, W,samplingcount,dataset_name)
                    else:
                        dataset_name=dataset_name+str(j)
                        new_X, new_y = SPECIAL1(dfx, dfy, W1,samplingcount,dataset_name)
                X1,y1=np.array(new_X),np.array(new_y)
            classifier.fit(X1,y1)
            X2,y2=X[test],y[test]
            ypred = classifier.predict(X2)
            yprob=classifier.predict_proba(X2)
            if classiferindex==1 or classiferindex==2 or classiferindex==3 or classiferindex==4:
                yprob=yprob.toarray()
            Macro.append(metrics.f1_score(y2, ypred,average='macro'))
            Micro.append(metrics.f1_score(y2, ypred,average='micro'))
            rankingloss.append(metrics.label_ranking_loss(y2,yprob))                     
            macropr.append(metrics.average_precision_score(y2,yprob,average='macro'))
            macroaucroc.append(metrics.roc_auc_score(y2,yprob,average='macro'))  
            Avgpr.append(metrics.average_precision_score(y2,yprob,average='samples'))
            Hamming_loss.append(metrics.hamming_loss(y2, ypred))  
    Avgpr=[a for a in Avgpr if a==a]
    
    MacroF=sum(Macro)/len(Macro)
#     print(Macro)
    MicroF=sum(Micro)/len(Micro)
    MacroAUCROC=sum(macroaucroc)/len(macroaucroc)
#     print(macroaucroc)
    MacroAUCPR=sum(macropr)/len(macropr)
    RankLoss=sum(rankingloss)/len(rankingloss)
    hamming=sum(Hamming_loss)/len(Hamming_loss)
    if len(Avgpr)!=0:
        Avgprecison=sum(Avgpr)/len(Avgpr)
    else:
        Avgprecison=0
    res2=(sum((i-MacroF)**2 for i in Macro)/len(Macro))**0.5
    res3=(sum((i-RankLoss)**2 for i in rankingloss)/len(rankingloss))**0.5
    res4=(sum((i-MacroAUCROC)**2 for i in macroaucroc)/len(macroaucroc))**0.5
    res6=(sum((i-MacroAUCPR)**2 for i in macropr)/len(macropr))**0.5
    if len(Avgpr)!=0:
        res7=(sum((i-Avgprecison)**2 for i in Avgpr)/len(Avgpr))**0.5
    else:
        res7=0
    MacroF=round(MacroF,4)
    MicroF=round(MicroF,4)
    MacroAUCROC=round(MacroAUCROC,4)
    MacroAUCPR=round(MacroAUCPR,4)
    RankLoss=round(RankLoss,4)
    Avgprecison=round(Avgprecison,4)
    hamming=round(hamming,4)
    res2=round(res2,4)
    res4=round(res4,4)
    res6=round(res6,4)
    res3=round(res3,4)
    res7=round(res7,4)
#     print(MacroF,MacroAUCROC,MacroAUCPR,RankLoss,res2,res4,res6,res3,res7)
    return MacroF,MacroAUCROC,MacroAUCPR,RankLoss,Avgprecison,res2,res4,res6,res3,res7

# BaseLine and Compared sampling method

In [65]:
path_to_arff_files = ["emotions.arff","scene.arff","yeast.arff", "rcv1subset1.arff","rcv1subset2.arff","rcv1subset3.arff","Corel5k.arff","yahoo-Business1","yahoo-Arts1.arff"]
label_counts = [6, 6,14,101,101,101,374,28,25]
select_feature=[1,1,1,0.01,0.01,0.01,1,0.01,0.01]
data_specific_sampling=[0.1,0.3,0.5,0.7,0.9]

for i, path_to_arff_file in enumerate(path_to_arff_files):
    X, y, feature_names, label_names = load_from_arff(
        path_to_arff_file,
        label_count=label_counts[i],
        label_location="end",
        load_sparse=False,
        return_attribute_definitions=True
    )
    X,feature_names=FeatureSelect(select_feature[i])
    y,label_names=LabelSelect()
    dataset_name = path_to_arff_file.split(".")[0] 
    print("----------------------------------------------------------------------------------")
    print(dataset_name)
    for c_idx in range(1,6):
        print (f"classifierid: {c_idx}")
        for m_idx in range(1, 9):
            if m_idx==6:
                sampling= data_specific_sampling2
            else:
                sampling= data_specific_sampling
            if m_idx<=3 or m_idx==7 or m_idx==8:
                MacroF,MacroAUCROC,MacroAUCPR,RankLoss,Avgprecison,res2,res4,res6,res3,res7=training(c_idx,m_idx,sp,dataset_name)
#                 print (f"sampling_method: {m_idx}")
#                 print(f"MacroF: {MacroF}, MacroAUCROC: {MacroAUCROC}, MacroAUCPR: {MacroAUCPR},RankLoss: {RankLoss},res2: {res2}, res4: {res4}, res6: {res6}, res3: {res3}")
                print(MacroF, MacroAUCROC, MacroAUCPR, RankLoss, res2, res4, res6, res3)
                continue
            else:
                MacroF_list = []
                MacroAUCROC_list = []
                MacroAUCPR_list = []
                RankLoss_list = []
                res2_list = []
                res4_list = []
                res6_list = []
                res3_list = []
#                 print (f"sampling_method: {m_idx}")
                for sp in sampling:
#                     print (f"sampling_count: {sp}")
                    MacroF,MacroAUCROC,MacroAUCPR,RankLoss,Avgprecison,res2,res4,res6,res3,res7=training(c_idx,m_idx,sp,dataset_name)
                    MacroF_list.append(MacroF)
                    MacroAUCROC_list.append(MacroAUCROC)
                    MacroAUCPR_list.append(MacroAUCPR)
                    RankLoss_list.append(RankLoss)
                    res2_list.append(res2)
                    res4_list.append(res4)
                    res6_list.append(res6)
                    res3_list.append(res3)
                max_MacroF = max(MacroF_list)
                max_MacroAUCROC = max(MacroAUCROC_list)
                max_MacroAUCPR = max(MacroAUCPR_list)
                min_RankLoss = min(RankLoss_list)
                res2_max = res2_list[MacroF_list.index(max_MacroF)]
                res4_max = res4_list[MacroAUCROC_list.index(max_MacroAUCROC)]
                res6_max = res6_list[MacroAUCPR_list.index(max_MacroAUCPR)]
                res3_min = res3_list[RankLoss_list.index(min_RankLoss)]
                print(max_MacroF, max_MacroAUCROC, max_MacroAUCPR, min_RankLoss, res2_max, res4_max, res6_max, res3_min)
#                 print(f"MacroF: {max(MacroF_list)}, MacroAUCROC: {max(MacroAUCROC_list)}, MacroAUCPR: {max(MacroAUCPR_list)}, RankLoss: {min(RankLoss_list)},res2: {res2_list[MacroF_list.index(max(MacroF_list))]}, res4: {res4_list[MacroAUCROC_list.index(max(MacroAUCROC_list))]}, res6: {res6_list[MacroAUCPR_list.index(max(MacroAUCPR_list))]}, res3: {res3_list[RankLoss_list.index(min(RankLoss_list))]}")

    print("----------------------------------------------------------------------------------")

----------------------------------------------------------------------------------
yahoo-Business1
classifierid: 1
0.1361 0.5745 0.0454 0.2262 0.0065 0.0049 0.002 0.0032
0.1352 0.5744 0.0449 0.2263 0.0076 0.0052 0.0028 0.0022
0.135 0.5743 0.0448 0.2262 0.0062 0.004 0.0021 0.0029
0.13 0.5712 0.0435 0.2272 0.0066 0.0046 0.0027 0.0031
0.1321 0.5723 0.0444 0.2264 0.0041 0.003 0.0014 0.0024
0.132 0.5705 0.0444 0.2272 0.0067 0.0043 0.0022 0.0035
0.1171 0.5737 0.0472 0.2255 0.0065 0.003 0.0026 0.0024
0.1321 0.5723 0.0444 0.2264 0.0041 0.003 0.0014 0.0024
classifierid: 2
0.1187 0.6326 0.1132 0.0556 0.0104 0.0072 0.0044 0.004
0.129 0.6397 0.1141 0.057 0.0118 0.0044 0.0053 0.0038
0.1282 0.6388 0.1153 0.0575 0.0128 0.0043 0.0054 0.0042
0.1629 0.6405 0.1147 0.0566 0.0099 0.0089 0.0079 0.0035
0.129 0.6305 0.1076 0.0581 0.0094 0.0065 0.0073 0.0034
0.1106 0.6284 0.1112 0.0559 0.0126 0.0055 0.0046 0.0036
0.1041 0.6354 0.1121 0.0582 0.0086 0.0057 0.0052 0.0031
0.129 0.6305 0.1076 0.0581 0.0094 0.0065 0

# Our method

In [ ]:
paracombine=[[2**(0),2**(-4)],[2**(0),2**(-2)],[2**(1),2**(-2)],[2**(2),2**(0)],[2**(3),2**(0)],[2**(3),2**(1)],[2**(4),2**(0)],[2**(4),2**(1)]]
# 'alpha'=lamda1,'beta'=lamda2,not give all parameter combinations
path_to_arff_files = ["emotions.arff","scene.arff","yeast.arff", "rcv1subset1.arff","rcv1subset2.arff","rcv1subset3.arff","Corel5k.arff","yahoo-Business1","yahoo-Arts1.arff"]
label_counts = [6, 6,14,101,101,101,374,28,25]
select_feature=[1,1,1,0.01,0.01,0.01,1,0.01,0.01]
data_specific_sampling=[0.05,0.1,0.3,0.5,0.7]
for i, path_to_arff_file in enumerate(path_to_arff_files):
    X, y, feature_names, label_names = load_from_arff(
        path_to_arff_file,
        label_count=label_counts[i],
        label_location="end",
        load_sparse=False,
        return_attribute_definitions=True
    )
    X,feature_names=FeatureSelect(select_feature[i])
    y,label_names=LabelSelect()
    dataset_name = path_to_arff_file.split(".")[0] 
    print(dataset_name)
    for c_idx in range(1,5):
        results_list=[]
        print (f"classifierid: {c_idx}")
        MacroF_list = []
        MacroAUCROC_list = []
        MacroAUCPR_list = []
        RankLoss_list = []
        res2_list = []
        res4_list = []
        res6_list = []
        res3_list = []
        for j in paracombine:
            optmParameter = {
                            'alpha':j[0], 
                            'beta': j[1],  
                            'gamma': 0.01,  
                            'lamda': 2**(4),  
                            'lamda2': 2**(6), 
                            'maxIter': 20, 
                            'minimumLossMargin': 0.01, 
                            'bQuiet': 1
                            }
            dataname=dataset_name+str(j[0])+str(j[1])
#             print (f"'alpha': {alpha_id},'beta': {beta_id}")
            for sp in data_specific_sampling:
                MacroF,MacroAUCROC,MacroAUCPR,RankLoss,Avgprecison,res2,res4,res6,res3,res7=training(c_idx,11,sp,dataname,optmParameter)                        
                MacroF_list.append(MacroF)
                MacroAUCROC_list.append(MacroAUCROC)
                MacroAUCPR_list.append(MacroAUCPR)
                RankLoss_list.append(RankLoss)
                res2_list.append(res2)
                res4_list.append(res4)
                res6_list.append(res6)
                res3_list.append(res3)
                result_dict = { 
                    "Classifier ID": c_idx,
                    'alpha':j[0], 
                    'beta': j[1], 
                    "Sampling Rate": sp,
                    "Macro F1": MacroF,
                    "Macro AUC ROC": MacroAUCROC,
                    "Macro AUC PR": MacroAUCPR,
                    "Rank Loss": RankLoss,
                    "std1":res2,
                    "std2":res4,
                    "std3":res6,
                    "std4":res3,
                }
                results_list.append(result_dict)
        max_MacroF = max(MacroF_list)
        max_MacroAUCROC = max(MacroAUCROC_list)
        max_MacroAUCPR = max(MacroAUCPR_list)
        min_RankLoss = min(RankLoss_list)
        res2_max = res2_list[MacroF_list.index(max_MacroF)]
        res4_max = res4_list[MacroAUCROC_list.index(max_MacroAUCROC)]
        res6_max = res6_list[MacroAUCPR_list.index(max_MacroAUCPR)]
        res3_min = res3_list[RankLoss_list.index(min_RankLoss)]
        print(max_MacroF, max_MacroAUCROC, max_MacroAUCPR, min_RankLoss, res2_max, res4_max, res6_max, res3_min)
#         print(f"MacroF: {max(MacroF_list)}, MacroAUCROC: {max(MacroAUCROC_list)}, MacroAUCPR: {max(MacroAUCPR_list)}, RankLoss: {min(RankLoss_list)},res2: {res2_list[MacroF_list.index(max(MacroF_list))]}, res4: {res4_list[MacroAUCROC_list.index(max(MacroAUCROC_list))]}, res6: {res6_list[MacroAUCPR_list.index(max(MacroAUCPR_list))]}, res3: {res3_list[RankLoss_list.index(min(RankLoss_list))]}")      
    results_df = pd.DataFrame(results_list)
    results_df.to_csv(f"{dataset_name}_{c_idx}.csv", index=False)
print("----------------------------------------------------------------------------------")

scene
classifierid: 1
0.5945 0.7534 0.441 0.447 0.0067 0.0102 0.0072 0.0176
classifierid: 2
0.7323 0.926 0.775 0.0868 0.0073 0.0028 0.0045 0.0031
classifierid: 3
0.5979 0.7555 0.4448 0.4248 0.013 0.0087 0.0122 0.015
classifierid: 4


result